# Fase 1: Importar las dependencias


In [1]:
import numpy as np
import math
import re                     #expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup #para trabajar con txt en diferentes formatos


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
#esto es solamente en GoogleColab, es para cargar la última TF

from tensorflow.keras import layers
import tensorflow_datasets as tfds #de aquí tomaremos el 'tokenizador'

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
rev = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Analisis de sentimientos reviews/peliculas.tsv', sep='\t')

In [5]:
rev.head(5)

,etiqueta,revision
0,pos,the happy bastard's 30-second review \r\nbig d...
1,neg,hollywood never fails to astound me . \r\never...
2,neg,"this is your definitive "" hollywood "" movie , ..."
3,pos,for those of us who weren't yet born when the ...
4,pos,NaN


In [6]:
#cols = ["sentiment", "text"]
#train_data = pd.read_csv(
#    '/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Analisis de sentimientos reviews/peliculas.tsv',
#    header=None,
#    names=cols,
#    engine="python",
#    encoding="latin1" #genralmente estandar para lenguas occidentales romances
#)
#test_data = pd.read_csv(
#    '/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Analisis de sentimientos Twitter/test data.csv',
#    header=None,
#    names=cols,
#    engine="python",
#    encoding="latin1"
#)

In [7]:
#train_data.head(5)

El conjunto de datos de testing tiene 3 etiquetas diferentes (una negativa, una positiva y una neutra), mientras que el conjunto de datos de entrenamiento tiene solo dos, por lo que no usaremos el archivo de testing y dividiremos el archivo de entrenamiento más tarde nosotros mismos.

## Pre Procesado

### Limpieza

In [8]:
rev.head(5)

,etiqueta,revision
0,pos,the happy bastard's 30-second review \r\nbig d...
1,neg,hollywood never fails to astound me . \r\never...
2,neg,"this is your definitive "" hollywood "" movie , ..."
3,pos,for those of us who weren't yet born when the ...
4,pos,NaN


In [9]:
rev.shape

(1473, 2)

In [10]:
rev = rev.dropna()

In [11]:
rev.shape

(1444, 2)

In [12]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    
    # Eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #https://regexr.com/ --> loquizar cualquier cosa que empiece con @+lo que sea y se sutituye por ' '

    # Eliminamos los links de las URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #el ? significa que el caracter anterior es opcional
    
    # Nos quedamos solamente con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet) #eliminar todo menos va con el ^ y pongo en []
    
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    
    return tweet

In [13]:
data_clean = [clean_tweet(tweet) for tweet in rev.revision]

In [14]:
set(rev.etiqueta)

{'neg', 'pos'}

In [15]:
#Para cambiar el 4 a 1
data_labels = rev.etiqueta.values
data_labels[data_labels == 'pos'] = 1
data_labels[data_labels == 'neg'] = 0
data_labels

array([1, 0, 0, ..., 1, 1, 1], dtype=object)

In [16]:
data_clean

Output hidden; open in https://colab.research.google.com to view.

### Tokenización

In [17]:
#De una lista de caracteres pasaremos a una lista de números

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16 #tamaño del vocabulario, en este casos las 65000 más frecuentes. Las menos frecuentes dificilmente la IA las pueda interpretar.
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

### Padding

In [18]:
#Padding --> Agrega CEROS hasta igualar todas las frases en su extención

MAX_LEN = max([len(sentence) for sentence in data_inputs]) #max busca la frase más larga
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Dividimos en los conjuntos de entrenamiento y de testing


In [37]:
#Esto es porque los Tweet estan ordenados 800mil neg y 800mil pos, sacaremos 8000 al azar de la primera mitad y 8000 de la segunda mitad

test_idx = np.random.randint(0, 1444, 1000) #los random de los neg
test_idx = np.concatenate((test_idx, test_idx-1)) # los pos se suman al mismo numero de los neg


In [38]:
test_inputs = data_inputs[test_idx] #txt tokenizados y padeado para entrenar la red neuronal
test_labels = data_labels[test_idx] #etiquetas 0 y 1 para entrenar la red neuronal
train_inputs = np.delete(data_inputs, test_idx, axis=0) #lo mismo pero le saco los utilizados arriba
train_labels = np.delete(data_labels, test_idx) #sin axis porque la etiqueta no es una matriz como lo son los txt


In [39]:
print(test_inputs.shape)
print(test_labels.shape)
print(train_inputs.shape)
print(train_labels.shape)

(2000, 2617)
(2000,)
(364, 2617)
(364,)


In [40]:
print(type(test_inputs))
print(type(test_labels))
print(type(train_inputs))
print(type(train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [41]:
 train_inputs= np.asarray(train_inputs).astype(np.float32)
 train_labels= np.asarray(train_labels).astype(np.float32)
 test_inputs= np.asarray(test_inputs).astype(np.float32)
 test_labels= np.asarray(test_labels).astype(np.float32)

# Fase 3: Construción del modelo

In [42]:
#Aquí comienza la CONVULSION, es decir, la aplicación de los filtros

class DCNN(tf.keras.Model): #instanciamos una clase
    
    def __init__(self,
                 vocab_size, #tamaño del vocab, dado por el tokenizador
                 emb_dim=128, #cada palabra será resumida a un numero de 128 caraceteres
                 nb_filters=50, #se pondrán 50 filtros de 4, 3 y 2 palabras
                 FFN_units=512, #número de neuronas de la capa oculta
                 nb_classes=2, #número de categorias de salida, en este caso son 2 por o neg
                 dropout_rate=0.1, #se usa para el entrenamiento, algunas neuronas se apagan para que no aprendan todas juntas, en cada vuelta el 10% se apagará 
                 training=False, #en la fase de predicción las neuronas funcionan todas
                 name="dcnn"): #le damos un nombre al modelo
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim) #tamaño del vocab
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2, #filtro de 2 palabras
                                    padding="valid",
                                    activation="relu") #cdo se activa la neurona? con esto que anda bien, es automático
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,#filtro de 3 palabras
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4, #filtro de 4 palabras
                                      padding="valid",
                                      activation="relu")
#El pooling comprimirá la info        
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate) #esto es para que no colapse la red
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")#para clasificacion binaria, si hubiera más categorias no podría usarse
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")#para pasar de 2 categorias/binaria a más de dos
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Paso 4: Aplicación

## Configuración

In [43]:
#concretizar todas las variables y parametros que se crearon arriba

VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200 #cada palabra se identificara con 200 cordenadas
NB_FILTERS = 100 #numero de filtros de la red
FFN_UNITS = 256 #numero de unidades de feet fowards en la capa oculta
NB_CLASSES = 2 #len(set(train_labels)) --> según las etiquetas de entrenamiento

DROPOUT_RATE = 0.2 #20% de las neuronas se apagan en el entrenamiento

BATCH_SIZE = 32 #se probará de 32 en 32 elementos = tweets
NB_EPOCHS = 10 # el data set se iterará 5 veces para mejorar la precisión

## Entrenamiento

In [44]:
#se creea el OBJETO Deep Convutional Neuronal Network
#se le pasa la configuración anterior

Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [45]:
#¿qué haremos cdo el proceso de clasificacion es bianario?

if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

#si no es binario se resuelve de la siguiente manera:

else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [46]:
#Esto es para que si se corta la sesión de GoogleColab y no arrancar desde el principio
#SIRVE PARA TENSOR FLOW

checkpoint_path = "/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Analisis de sentimientos reviews/ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


In [47]:
#Ajustar los datos

Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)

ckpt_manager.save()

Epoch 1/10
12/12 [==============================] - 2s 71ms/step - loss: 0.0251 - accuracy: 0.9863
Epoch 2/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0127 - accuracy: 0.9945
Epoch 3/10
12/12 [==============================] - 1s 51ms/step - loss: 0.0127 - accuracy: 0.9945
Epoch 4/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0122 - accuracy: 0.9945
Epoch 5/10
12/12 [==============================] - 1s 51ms/step - loss: 0.0120 - accuracy: 0.9945
Epoch 6/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0118 - accuracy: 0.9945
Epoch 7/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0117 - accuracy: 0.9945
Epoch 8/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0117 - accuracy: 0.9945
Epoch 9/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0115 - accuracy: 0.9945
Epoch 10/10
12/12 [==============================] - 1s 50ms/step - loss: 0.0115 - accuracy: 0.9945


'/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Analisis de sentimientos reviews/ckpt/ckpt-3'

## Evaluación

In [48]:
#aqui aplica lo entrenado a tweets que no ha leido antes

results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

63/63 [==============================] - 1s 19ms/step - loss: 0.0350 - accuracy: 0.9810
[0.034971509128808975, 0.9810000061988831]


Aquí se prueba con Tweets "artificiales", con el entrenamiento anterior

In [49]:
Dcnn(np.array([tokenizer.encode("The film is boring")]), training=False).numpy()

array([[0.33320624]], dtype=float32)

In [ ]:
tokenizer.encode("You are so funny")

[43856, 43878, 4594, 22, 42, 691]